In [95]:
import pyodbc
import re
import json

import numpy as np
import pandas as pd
import tensorflow as tf

from sqlalchemy import create_engine
from sklearn.metrics.pairwise import cosine_similarity

from collections import defaultdict
from gensim import corpora, models, similarities
from gensim.similarities import SoftCosineSimilarity, SparseTermSimilarityMatrix, WordEmbeddingSimilarityIndex
from gensim.models.word2vec import Word2Vec

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from googletrans import Translator
from deep_translator import GoogleTranslator

from FlaskApp.transform import *

from job_model import JobModel
from app_model import AppModel

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [96]:
translator = Translator(service_urls=['translate.googleapis.com'])
factory = StemmerFactory()
stemmer = factory.create_stemmer()

rootwords = [i[0] for i in pd.read_csv('data/rootwords.txt').values]

with open('data/slangwords.json', 'r') as file:
    slangwords = json.load(file)

with open('data/englishwords.json', 'r') as file:
    englishwords = json.load(file)

with open('data/slangjobs.json', 'r') as file:
    slangjobs = json.load(file)

In [97]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

In [98]:
user = 'huda'
password = 'Vancha12'
host = '127.0.0.1'
port = 1433
database = 'HRSystemDB'


def get_connection():         
    return create_engine(
        url=f"mssql+pyodbc://{user}:{password}@{host}:{port}/{database}?driver=SQL Server",
    )

engine = get_connection()
conn = engine.connect()

In [99]:
applicant_id = 31790

df_job = pd.DataFrame(engine.execute(
    """
    SELECT Job.JobID, Job.UsiaMax, Job.SalaryMin, Job.SalaryMax, City.Name AS CityName, Province.Name AS ProvinceName, EducationLevel.EducationLevelName, Major.MajorName, Job.DriverLicenseType, Job.UsingGlasses, Job.Gender, Job.MaritalStatus, Job.JobTitle, FunctionPosition.FunctionPositionName, Job.Description, Job.Requirement
    FROM (((((Job
    RIGHT JOIN FunctionPosition ON Job.FunctionPositionID = FunctionPosition.FunctionPositionID)
    RIGHT JOIN EducationLevel ON Job.EducationLevelID = EducationLevel.EducationLevelID)
    RIGHT JOIN City ON Job.CityID = City.CityID)
    RIGHT JOIN Province ON Job.ProvinceID = Province.ProvinceID)
    RIGHT JOIN Major ON Job.MajorID = Major.MajorID)
    """
))

df_function = pd.DataFrame(engine.execute(
    """
    SELECT FunctionPositionID, FunctionPositionName
    FROM FunctionPosition
    """
))

df_education = pd.DataFrame(engine.execute(
    """
    SELECT EducationLevelID, EducationLevelName
    FROM EducationLevel
    """
))

df_city = pd.DataFrame(engine.execute(
    """
    SELECT CityID, Name AS CityName
    FROM City
    """
))

df_province = pd.DataFrame(engine.execute(
    """
    SELECT ProvinceID, Name AS ProvinceName
    FROM Province
    """
))

df_major = pd.DataFrame(engine.execute(
    """
    SELECT MajorID, MajorName
    FROM Major
    """
))

df_applicant = pd.DataFrame(engine.execute(
    f"""
    SELECT Applicant.ApplicantID, Applicant.Dob, Applicant.ExpectedSalary, City.Name AS CityName, Province.Name AS ProvinceName, Applicant.DriverLicenseType, Applicant.IsUsingGlasses, Applicant.Gender, Applicant.MaritalStatus, Applicant.Strengthness
    FROM (((Applicant
    RIGHT JOIN City ON Applicant.CurrentAddressCityID = City.CityID)
    RIGHT JOIN Province ON Applicant.CurrentAddressProvinceID = Province.ProvinceID)
    LEFT JOIN Pipeline ON Applicant.ApplicantID = Pipeline.ApplicantID)
    """
))

df_applicant_education = pd.DataFrame(engine.execute(
    f"""
    SELECT ApplicantEducation.ApplicantID, ApplicantEducation.DateStart, ApplicantEducation.DateEnd, EducationLevel.EducationLevelName, Major.MajorName
    FROM (((ApplicantEducation
    RIGHT JOIN EducationLevel ON ApplicantEducation.EducationLevelID = EducationLevel.EducationLevelID)
    RIGHT JOIN Major ON ApplicantEducation.MajorID = Major.MajorID)
    LEFT JOIN Pipeline ON ApplicantEducation.ApplicantID = Pipeline.ApplicantID)
    """
))

df_applicant_experience = pd.DataFrame(engine.execute(
    f"""
    SELECT ApplicantExperience.ApplicantID, ApplicantExperience.DateFrom, ApplicantExperience.DateTo, ApplicantExperience.Position, ApplicantExperience.JobDescription
    FROM (ApplicantExperience
    LEFT JOIN Pipeline ON ApplicantExperience.ApplicantID = Pipeline.ApplicantID)
    """
))

In [100]:
job_str = ['CityName', 'ProvinceName', 'EducationLevelName', 'MajorName', 'DriverLicenseType', 'Gender', 'MaritalStatus', 'JobTitle', 'FunctionPositionName', 'Description', 'Requirement']
job_num = ['UsiaMax', 'SalaryMin', 'SalaryMax']
job_bol = ['UsingGlasses']

'''general'''
df_job.set_index(['JobID'], inplace=True)
df_job.fillna('', inplace=True)

'''str'''
df_job[job_str] = df_job[job_str].applymap(str.lower)
df_job.replace('none', '', inplace=True)

In [7]:
df_job.Description = df_job.Description.map(
    clean_text
).apply(
    lambda x: change_words(slangwords, x)
).apply(
    lambda x: change_words(englishwords, x)
).apply(
    lambda x: change_words(slangwords, x)
).map(
    stemmer.stem
)

In [8]:
df_job.Requirement = df_job.Requirement.map(
    clean_text
).apply(
    lambda x: change_words(slangwords, x)
).apply(
    lambda x: change_words(englishwords, x)
).apply(
    lambda x: change_words(slangwords, x)
).map(
    stemmer.stem
)

In [9]:
corpus = list(set(' '.join([i for i in df_job.Requirement.values]).split()))
corpus = [i for i in corpus if i not in rootwords]
corpus.sort()

In [12]:
for i in corpus:
    if i not in rootwords:
        print(i)

adwords
aldo
alphard
aplication
atu
audisi
auravita
berpengalan
campaignare
cctv
clearil
co
competency
cslr
delivering
dki
dna
drafter
drone
drupal
ds
effect
estimator
excell
fbm
firebase
foodteria
freshgraduated
gadget
gcg
gm
grapis
handphone
hrga
hsk
html
http
ifca
instagramable
intrapersonal
io
iso
ivena
javascript
jquery
json
kta
lainnnya
litigasi
maks
matic
mcse
mempu
mengoptimasi
merapihkan
mikrotik
mou
multi
myob
nat
novianti
od
paced
pb
pc
pdf
penempatakan
perjalalanan
php
pkwt
pkwtt
placement
pods
porsi
powerpoint
preschool
programmer
proposional
proxy
qlikview
rekon
respon
router
sbb
sd
sejeniis
seo
serentak
sitac
skalatis
sketchup
skill
sma
smartphone
smp
smu
solving
sony
spesialisasi
sq
sqlserver
ssas
ssis
staadpro
stm
strategic
sudirman
supir
taiwan
tamb
tekana
teknisi
tekut
ten
terstuktur
thingking
tr
treassury
usi
uud
vb
vegas
videotron
wafel
waze
willingness
winbox
wp
yo
zahir


In [13]:
# TODO: cleansing requirement belum selesai
# TODO: buat istilah paten mengenai skill dan keperluan tertentu

In [10]:
df_job.JobTitle = df_job.JobTitle.map(
    remove_insideparentheses
).map(
    remove_standalonesymbols
).map(
    remove_morespace
).apply(
    lambda x: change_words(slangjobs, x)
)
df_job.FunctionPositionName = df_job.FunctionPositionName.map(
    remove_insideparentheses
).map(
    remove_standalonesymbols
).map(
    remove_morespace
).apply(
    lambda x: change_words(slangjobs, x)
)

In [10]:
corpus = list(set(' '.join([i for i in df_job.Description.values]).split()))
corpus = [i for i in corpus if i not in rootwords]
corpus.sort()

In [ ]:
# english_corpus = []
# for i in corpus:
#     translated = translator.detect(i)
#     if translated.lang == 'en' and translated.confidence > .75:
#         english_corpus.append(i)

# indonesian_corpus = []
# for i in corpus:
#     translated = translator.detect(i)
#     if translated.lang == 'in' and translated.confidence > .75:
#         indonesian_corpus.append(i)

# another_corpus = []

# for i in corpus:
#     if i not in english_corpus and i not in indonesian_corpus:
#         another_corpus.append(i)

In [ ]:
# translator = GoogleTranslator(source='en', target='id')

# corpus.sort()

# corpused = {}
# for original in corpus:
#     translated = translator.translate(original)
#     corpused[original] = str.lower(translated)

# no_translate = []
# for key in corpused:
#     if key == corpused[key]:
#         no_translate.append(key)

# for key in no_translate:
#     del corpused[key]

# slang_2 = []

# for i,j in zip(slang, stemmed):
#     if i == j:
#         slang_2.append(i)

In [102]:
'''applicant'''
df_applicant = df_applicant.dropna(subset=['ApplicantID'])
df_applicant.ApplicantID = df_applicant.ApplicantID.astype(int)
df_applicant = df_applicant.drop_duplicates()
df_applicant = df_applicant.fillna('')
# age column
df_applicant['Age'] = pd.to_datetime(
    df_applicant.Dob.map(pick_date).apply(lambda x: filter_date(x, 1958, 2006))
).map(get_age)

df_applicant.drop(columns=['Dob'], inplace=True)

df_applicant.Age = df_applicant.Age.fillna(0).astype(int)
'''education'''
df_applicant_education = df_applicant_education.fillna('')
# datetime column
df_applicant_education.DateStart = pd.to_datetime(
    df_applicant_education.DateStart.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)
df_applicant_education.DateEnd = pd.to_datetime(
    df_applicant_education.DateEnd.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)
df_applicant_education = df_applicant_education[~(df_applicant_education.DateStart.isna()) & ~(df_applicant_education.DateEnd.isna())]
df_applicant_education = df_applicant_education.sort_values('DateStart').groupby(['ApplicantID']).agg('last')
df_applicant_education.drop(columns=['DateStart', 'DateEnd'], inplace=True)
'''experience'''
df_applicant_experience = df_applicant_experience.fillna('')

# datetime column
df_applicant_experience.DateFrom = pd.to_datetime(
    df_applicant_experience.DateFrom.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)
df_applicant_experience.DateTo = pd.to_datetime(
    df_applicant_experience.DateTo.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)
df_applicant_experience = df_applicant_experience[~(df_applicant_experience.DateFrom.isna()) & ~(df_applicant_experience.DateTo.isna())]

# add YearsOfExperience column
df_applicant_experience['YearsOfExperience'] = substract_months(
    df_applicant_experience.DateFrom, df_applicant_experience.DateTo
)
df_applicant_experience = df_applicant_experience.sort_values('DateFrom').groupby(['ApplicantID']).agg({
    'DateFrom': 'last',
    'DateTo': 'last',
    'JobDescription': ' '.join,
    'Position': ' '.join,
    'YearsOfExperience': 'sum',
})
df_applicant_experience.drop(columns=['DateFrom', 'DateTo'], inplace=True)
'''merge'''
df_applicant = pd.merge(df_applicant, df_applicant_experience, on=['ApplicantID'])
df_applicant = pd.merge(df_applicant, df_applicant_education, on=['ApplicantID'])

C:\Users\eats\AppData\Local\Temp\ipykernel_6088\1545315867.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_applicant.ApplicantID = df_applicant.ApplicantID.astype(int)


In [103]:
'''PRE-PROCESSING APPLICANT'''
df_applicant.set_index(['ApplicantID'], inplace=True)

app_str = ['CityName', 'ProvinceName', 'EducationLevelName', 'MajorName', 'DriverLicenseType', 'Gender', 'MaritalStatus', 'Position', 'JobDescription', 'Strengthness']
app_num = ['Age', 'ExpectedSalary', 'YearsOfExperience']
app_bol = ['IsUsingGlasses']

df_applicant = df_applicant[app_num + app_bol + app_str]

'''str'''
df_applicant[app_str] = df_applicant[app_str].applymap(str.lower)

In [11]:
df_applicant.JobDescription = df_applicant.JobDescription.map(
    clean_text
).apply(
    lambda x: change_words(slangwords, x)
).apply(
    lambda x: change_words(englishwords, x)
).apply(
    lambda x: change_words(slangwords, x)
).map(
    stemmer.stem
)

df_applicant.Strengthness = df_applicant.Strengthness.map(
    clean_text
).apply(
    lambda x: change_words(slangwords, x)
).apply(
    lambda x: change_words(englishwords, x)
).apply(
    lambda x: change_words(slangwords, x)
).map(
    stemmer.stem
)

df_applicant.Position = df_applicant.Position.map(
    remove_insideparentheses
).map(
    remove_standalonesymbols
).map(
    remove_morespace
).apply(
    lambda x: change_words(slangjobs, x)
)

c:\Users\eats\miniconda3\envs\p1\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [104]:
# concat
df_applicant.JobDescription = df_applicant.JobDescription.str.cat(
    df_applicant.Strengthness, sep=' '
)
df_applicant.rename(columns={'JobDescription': 'DescriptionStrengthness'}, inplace=True)
df_applicant.drop(columns=['Strengthness'], inplace=True)

'''bool'''
df_applicant.IsUsingGlasses = df_applicant.IsUsingGlasses.astype(str).map(str.lower)

In [105]:
# concatenate JobTitle and FunctionPositionName to be textual feature together
df_job.JobTitle = df_job.JobTitle.str.cat(
    df_job.FunctionPositionName, sep=' '
)
df_job.rename(columns={'JobTitle': 'JobTitlePosition'}, inplace=True)
df_job.drop(columns=['FunctionPositionName'], inplace=True)

# concatenate Description and Requirement to be textual feature together
df_job.Description = df_job.Description.str.cat(
    df_job.Requirement, sep=' '
)
df_job.rename(columns={'Description': 'DescriptionRequirement'}, inplace=True)
df_job.drop(columns=['Requirement'], inplace=True)

df_job = df_job[~(df_job.DescriptionRequirement == ' ')]


In [106]:
df_job['Texts'] = df_job.JobTitlePosition + ' ' + df_job.DescriptionRequirement
df_applicant['Texts'] = df_applicant.Position + ' ' + df_applicant.DescriptionStrengthness

In [107]:
import gensim.downloader as api
from gensim.models import doc2vec

# get dataset
# dataset = api.load("text8")
dataset = list(df_job.Texts.apply(lambda x: x.split()).values)
data =[]
for w in dataset:
    data.append(w)

# To train the model we need a list of tagged documents
def tagged_document(list_of_ListOfWords):
    for x, ListOfWords in enumerate(list_of_ListOfWords):
        yield doc2vec.TaggedDocument(ListOfWords, [x])

# training data
data_train = list(tagged_document(data))

In [108]:
# Initialize the model
d2v_model = doc2vec.Doc2Vec(vector_size=120, min_count=10, epochs=90)

# build the vocabulary
d2v_model.build_vocab(data_train)

# Train Doc2Vec model
d2v_model.train(data_train, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

2023-04-18 16:56:35,207 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d120,n5,w5,mc10,s0.001,t3>', 'datetime': '2023-04-18T16:56:35.207108', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-04-18 16:56:35,209 : INFO : collecting all words and their counts
2023-04-18 16:56:35,209 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2023-04-18 16:56:35,253 : INFO : collected 12888 word types and 1006 unique tags from a corpus of 1006 examples and 393745 words
2023-04-18 16:56:35,254 : INFO : Creating a fresh vocabulary
2023-04-18 16:56:35,263 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=10 retains 2639 unique words (20.48% of original 12888, drops 10249)', 'datetime': '2023-04-18T16:56:35.263122', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 

In [110]:
d2v_model.similarity_unseen_docs(df_job.JobTitlePosition.loc[1607].split(), df_applicant.Position.loc[39348].split())

0.22350791

In [57]:
import multiprocessing

cores = multiprocessing.cpu_count()

In [75]:
texts = df_job.DescriptionRequirement.map(str.split).tolist()

In [76]:
model = models.Word2Vec(min_count=20, window=2, vector_size=300, sample=6e-5,  alpha=0.03,  min_alpha=0.0007,  negative=20, workers=cores-1)

2023-04-18 16:26:48,643 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2023-04-18T16:26:48.643431', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


In [77]:
model.build_vocab(texts, progress_per=10000)

2023-04-18 16:26:49,780 : INFO : collecting all words and their counts
2023-04-18 16:26:49,781 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-04-18 16:26:49,791 : INFO : collected 1624 word types from a corpus of 104133 raw words and 1005 sentences
2023-04-18 16:26:49,792 : INFO : Creating a fresh vocabulary
2023-04-18 16:26:49,795 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=20 retains 609 unique words (37.50% of original 1624, drops 1015)', 'datetime': '2023-04-18T16:26:49.795381', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'prepare_vocab'}
2023-04-18 16:26:49,795 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=20 leaves 98239 word corpus (94.34% of original 104133, drops 5894)', 'datetime': '2023-04-18T16:26:49.795381', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (

In [78]:
model.train(texts, total_examples=model.corpus_count, epochs=30, report_delay=1)

2023-04-18 16:26:51,422 : INFO : Word2Vec lifecycle event {'msg': 'training model with 15 workers on 609 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2 shrink_windows=True', 'datetime': '2023-04-18T16:26:51.421296', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
2023-04-18 16:26:51,454 : INFO : EPOCH 0: training on 104133 raw words (19157 effective words) took 0.0s, 743808 effective words/s
2023-04-18 16:26:51,485 : INFO : EPOCH 1: training on 104133 raw words (19146 effective words) took 0.0s, 788953 effective words/s
2023-04-18 16:26:51,521 : INFO : EPOCH 2: training on 104133 raw words (19135 effective words) took 0.0s, 807234 effective words/s
2023-04-18 16:26:51,553 : INFO : EPOCH 3: training on 104133 raw words (19377 effective words) took 0.0s, 916569 effective words/s
2023-04-18 16:26:51,585 : INFO : EPOCH 4:

(576456, 3123990)

In [79]:
model.init_sims(replace=True)

C:\Users\eats\AppData\Local\Temp\ipykernel_6088\1992334471.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)
2023-04-18 16:26:53,910 : WARNING : destructive init_sims(replace=True) deprecated & no longer required for space-efficiency


In [94]:
model.wv.infe

KeyError: "Key 'teknologi hukum' not present"

In [90]:
df_applicant.DescriptionStrengthness.loc[39348].split()[0]

'riset'

In [34]:
df_job.JobTitlePosition.loc[3060]

'ios pengembang teknologi informasi'

In [35]:
df_applicant.Position.loc[13]

'teknologi informasi teknologi informasi teknologi informasi'

In [33]:
d2v_model.similarity_unseen_docs([df_job.JobTitlePosition.loc[1607]], [df_applicant.Position.loc[39348]])

0.015874676

In [23]:
df_applicant[df_applicant.Position.str.contains('ios')]

,Age,ExpectedSalary,YearsOfExperience,IsUsingGlasses,CityName,ProvinceName,EducationLevelName,MajorName,DriverLicenseType,Gender,MaritalStatus,Position,DescriptionStrengthness,Texts
ApplicantID,,,,,,,,,,,,,,
39348,25,6000000.0,0.833333,false,surabaya,jawa timur,s1,teknik informatika,c,male,single,ios pengembang,riset dan masalah validation ada umum dan doma...,ios pengembang riset dan masalah validation ad...


In [25]:
df_job[df_job.JobTitlePosition.str.contains('ios')]

,UsiaMax,SalaryMin,SalaryMax,CityName,ProvinceName,EducationLevelName,MajorName,DriverLicenseType,UsingGlasses,Gender,MaritalStatus,JobTitlePosition,DescriptionRequirement,Texts
JobID,,,,,,,,,,,,,,
1607,0.0,0.0,0.0,surabaya,jawa timur,d3,teknik informatika,,False,,,ios pengembang website pembuat program,kembang aplikasi ios dan integrasi dengan laya...,ios pengembang website pembuat program kembang...
2753,0.0,0.0,0.0,surabaya,jawa timur,d3,teknik informatika,,False,,,ios pengembang website pembuat program,kembang aplikasi ios dan integrasi dengan laya...,ios pengembang website pembuat program kembang...
2756,0.0,0.0,0.0,surabaya,jawa timur,d3,teknik informatika,,False,,,ios pengembang teknologi informasi,kembang aplikasi ios dan integrasi dengan laya...,ios pengembang teknologi informasi kembang apl...
3060,0.0,0.0,0.0,surabaya,jawa timur,d3,teknik komputer,,False,,,ios pengembang teknologi informasi,kembang aplikasi ios dan integrasi dengan laya...,ios pengembang teknologi informasi kembang apl...


In [28]:
df_applicant.Position

ApplicantID
3        penjualan eksekutif whole penjualan credit pem...
13       teknologi informasi teknologi informasi teknol...
21             java website pengembang android pengembang 
22                                asisten dosen supervisor
25                     pemula progammer progammer  analis 
                               ...                        
40027                                            pengemudi
40031                                     hukum kordinator
40033                                        hukum petugas
41402    senior data pelayanan resepsionis cum administ...
43028                                            pengemudi
Name: Position, Length: 3629, dtype: object